# Проблематика
В строительной отрасли взаимодействие между различными участниками проекта — проектировщиками, техническими заказчиками, генподрядчиками и инженерами-производственниками — часто осложняется из-за объемного документооборота, необходимости быстрого доступа к актуальным данным и сложностью анализа информации. Традиционные методы хранения и обмена данными, такие как бумажные документы или вручную введенные отчеты, замедляют рабочие процессы и увеличивают вероятность ошибок.

Эти проблемы ведут к увеличению затрат времени и ресурсов на согласование проектов, что в конечном итоге может задерживать сроки сдачи объектов и негативно влиять на качество выполненных работ.
## Цель работы бота
Цель разработки Telegram-бота заключается в создании удобного цифрового инструмента, который объединит участников строительных проектов и упростит их взаимодействие.

С помощью бота пользователи смогут:

- Собирать и хранить информацию по проекту, в том числе статус выполнения работ и обновления на этапах строительства.
- Анализировать данные за счет автоматического сбора и вывода информации в формате Excel, что позволит участникам проекта проводить регулярный анализ и делиться актуальными данными.
- Оптимизировать отчетность и обеспечить прозрачность процессов за счет автоматического формирования отчетов, доступных для всех участников проекта.
  
Таким образом, бот способствует повышению эффективности и прозрачности на всех этапах проекта, улучшает коммуникацию между участниками и помогает контролировать ключевые показатели в режиме реального времени.

## Импортирование всех модулей

In [1]:
import nest_asyncio
nest_asyncio.apply() # Add async to Jupyter Notebook

from telegram import Update, ReplyKeyboardMarkup, ReplyKeyboardRemove
from telegram.ext import Application, CommandHandler, CallbackContext, MessageHandler, filters, ContextTypes, ConversationHandler
import sqlite3
import openpyxl
from io import BytesIO

## Приветственное сообщение пользователю

In [2]:
welcome_message = """
Добро пожаловать в ПроектМенеджер! 🏗️

ПроектМенеджер — это ваш цифровой помощник в строительной отрасли. Этот бот позволяет удобно управлять проектами строительства, добавлять информацию о каждом объекте, обновлять данные, а также экспортировать отчеты в формате Excel для обмена и анализа.

Что умеет бот:

Создание новых строительных проектов с указанием ключевых параметров 🏢
Обновление информации о текущих проектах 🔄
Просмотр списка всех проектов 📋
Экспорт данных проектов в Excel для удобного анализа 📊

Кому будет полезен:

Проектировщикам
Генподрядчикам
Техническим заказчикам
Инженерам ПТО
Начните с команды /start, и бот поможет вам настроить учет ваших строительных объектов быстро и просто!

Команды:

/start — Начать работу с ботом и узнать доступные команды
/add_project — Создать новый проект
/update_project — Обновить существующий проект
/delete_project - Удалить существующий проект
/list_projects — Показать список всех проектов
/show_project_details - Показать детальное описание проекта
/export_project — Экспортировать проект в Excel
Готовы начать? Просто введите /start, и СтройПомощник проведет вас по всем этапам!
"""

## Реализация команды ```/start```

In [3]:
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await update.message.reply_text(welcome_message)

## Реализация команды ```/add_project```

In [4]:
# Шаги диалога
NAME, DESCRIPTION, LOCATION, START_DATE, END_DATE = range(5)

# Функция для добавления нового проекта в базу данных
def add_project_to_db(user_id, name, description, location, start_date, end_date):
    conn = sqlite3.connect('projects.db')
    cursor = conn.cursor()
    cursor.execute("INSERT INTO projects (user_id, name, description, location, start_date, end_date) VALUES (?, ?, ?, ?, ?, ?)",
                   (user_id, name, description, location, start_date, end_date))
    conn.commit()
    conn.close()

# Функция для старта команды /add_project
async def start_add_project(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    await update.message.reply_text("Введите имя проекта:")
    return NAME

# Функция для получения имени проекта
async def get_name(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    context.user_data['name'] = update.message.text  # Сохраняем имя в user_data
    await update.message.reply_text("Введите описание проекта:")
    return DESCRIPTION

# Функция для получения описания проекта
async def get_description(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    context.user_data['description'] = update.message.text  # Сохраняем описание
    await update.message.reply_text("Введите локацию (адрес) постройки объекта:")
    return LOCATION

# Функция для получения локации проекта
async def get_location(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    context.user_data['location'] = update.message.text  # Сохраняем локацию
    await update.message.reply_text("Введите дату начала проекта (например, 2024-01-20):")
    return START_DATE

# Функция для получения даты начала проекта
async def get_start_date(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    context.user_data['start_date'] = update.message.text  # Сохраняем дату начала
    await update.message.reply_text("Введите дату окончания проекта (например, 2024-12-31):")
    return END_DATE

# Функция для получения даты окончания проекта
async def get_end_date(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    context.user_data['end_date'] = update.message.text  # Сохраняем дату окончания

    # Get the previous user answers
    user_id = update.effective_user.id
    name = context.user_data['name']
    description = context.user_data['description']
    location = context.user_data['location']
    start_date = context.user_data['start_date']
    end_date = context.user_data['end_date']
    
    # Добавляем проект в базу данных
    add_project_to_db(user_id, name, description, location, start_date, end_date)

    # Подтверждаем добавление проекта
    await update.message.reply_text(f"Проект '{name}' успешно добавлен!")
    return ConversationHandler.END

# Функция для отмены добавления проекта
async def cancel(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    await update.message.reply_text("Процесс добавления проекта отменен.")
    return ConversationHandler.END

# Создаем ConversationHandler для команд /add_project
conversation_handler = ConversationHandler(
    entry_points=[CommandHandler('add_project', start_add_project)],
    states={
        NAME: [MessageHandler(filters.TEXT & ~filters.COMMAND, get_name)],
        DESCRIPTION: [MessageHandler(filters.TEXT & ~filters.COMMAND, get_description)],
        LOCATION: [MessageHandler(filters.TEXT & ~filters.COMMAND, get_location)],
        START_DATE: [MessageHandler(filters.TEXT & ~filters.COMMAND, get_start_date)],
        END_DATE: [MessageHandler(filters.TEXT & ~filters.COMMAND, get_end_date)],
    },
    fallbacks=[CommandHandler('cancel', cancel)],
)

## Реализация команды ```/list_projects```

In [5]:
# Define the function for the /list_projects command
async def list_projects(update: Update, context: CallbackContext):
    user_id = update.effective_user.id
    conn = sqlite3.connect("projects.db")
    cursor = conn.cursor()
    
    # Retrieve projects for the current user
    cursor.execute("SELECT name, description, location, start_date, end_date FROM projects WHERE user_id = ?", (user_id,))
    projects = cursor.fetchall()
    
    # If no projects are found
    if not projects:
        await update.message.reply_text("У вас пока что нет каких либо проектов ;(")
    else:
        # Format the list of projects
        project_list = "\n".join([f"🔹 {project[0]}" for project in projects])
        await update.message.reply_text(f"Вот список ваших проектов:\n\n{project_list}")
    
    conn.close()

## Реализация команды ```/update_project```

In [6]:
# Dialog state
SELECT_PROJECT, UPDATE_CHOICE, UPDATE_NAME, UPDATE_DESCRIPTION, UPDATE_LOCATION, UPDATE_START_DATE, UPDATE_END_DATE = range(7)

# Define the function for the /update_project command
async def update_project_start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    user_id = update.effective_user.id

    # Получение списка проектов пользователя
    conn = sqlite3.connect("projects.db")
    cursor = conn.cursor()
    cursor.execute("SELECT id, name FROM projects WHERE user_id = ?", (user_id,))
    projects = cursor.fetchall()
    conn.close()

    if not projects:
        await update.message.reply_text("У вас пока нет проектов для редактирования ;(")
        return ConversationHandler.END

    project_names = [project[1] for project in projects]
    context.user_data['project_ids'] = {project[1]: project[0] for project in projects}
    keyboard = [[name] for name in project_names]
    
    await update.message.reply_text(
        "Выберите проект, который хотите редактировать:",
        reply_markup=ReplyKeyboardMarkup(keyboard, one_time_keyboard=True)
    )
    return SELECT_PROJECT

# Handle a user project selection
async def select_project(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    project_name = update.message.text
    project_id = context.user_data['project_ids'].get(project_name)

    if not project_id:
        await update.message.reply_text("Проект не найден. Пожалуйста, выберите проект из списка.")
        return SELECT_PROJECT

    context.user_data['project_id'] = project_id
    await update.message.reply_text(
        """
        Что вы хотите обновить в проекте?
        
        Введите 'name' для изменения названия 
        Введите 'description' для изменения описания 
        Введите 'location' для изменения локации
        Введите 'start_date' для изменения даты начала строительства  
        Введите 'end_date' для изменения даты окончания строительства
        """,
        reply_markup=ReplyKeyboardRemove()
    )
    return UPDATE_CHOICE

# Обработчик для обновления названия или параметров проекта
async def update_project_choice(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    choice = update.message.text.lower()
    if choice == 'name':
        await update.message.reply_text("Введите новое название проекта:")
        return UPDATE_NAME
    elif choice == 'description':
        await update.message.reply_text("Введите новое описание проекта:")
        return UPDATE_DESCRIPTION
    elif choice == 'location':
        await update.message.reply_text("Введите новый адрес проекта:")
        return UPDATE_LOCATION
    elif choice == 'start_date':
        await update.message.reply_text("Введите новую дату начала строительства:")
        return UPDATE_START_DATE
    elif choice == 'end_date':
        await update.message.reply_text("Введите новую дату окончания строительства:")
        return UPDATE_END_DATE
    else:
        await update.message.reply_text("Неверный выбор. Пожалуйста, введите 'name', 'description', 'location', 'start_date' или 'end_date'.")
        return UPDATE_CHOICE

# Обработчик обновления названия проекта
async def update_project_name(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    new_name = update.message.text
    project_id = context.user_data['project_id']

    conn = sqlite3.connect("projects.db")
    cursor = conn.cursor()
    cursor.execute("UPDATE projects SET name = ? WHERE id = ?", (new_name, project_id))
    conn.commit()
    conn.close()

    await update.message.reply_text(f"Название проекта обновлено на '{new_name}'.")
    return ConversationHandler.END

async def update_project_description(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    new_description = update.message.text
    project_id = context.user_data['project_id']

    conn = sqlite3.connect("projects.db")
    cursor = conn.cursor()
    cursor.execute("UPDATE projects SET description = ? WHERE id = ?", (new_description, project_id))
    conn.commit()
    conn.close()

    await update.message.reply_text("Описание проекта обновлено.")
    return ConversationHandler.END

async def update_project_location(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    new_location = update.message.text
    project_id = context.user_data['project_id']

    conn = sqlite3.connect("projects.db")
    cursor = conn.cursor()
    cursor.execute("UPDATE projects SET location = ? WHERE id = ?", (new_location, project_id))
    conn.commit()
    conn.close()

    await update.message.reply_text("Адрес проекта обновлен.")
    return ConversationHandler.END

async def update_project_start_date(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    new_start_date = update.message.text
    project_id = context.user_data['project_id']

    conn = sqlite3.connect("projects.db")
    cursor = conn.cursor()
    cursor.execute("UPDATE projects SET start_date = ? WHERE id = ?", (new_start_date, project_id))
    conn.commit()
    conn.close()

    await update.message.reply_text("Начальная дата строительства объекта обновлена.")
    return ConversationHandler.END

async def update_project_end_date(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    new_end_date = update.message.text
    project_id = context.user_data['project_id']

    conn = sqlite3.connect("projects.db")
    cursor = conn.cursor()
    cursor.execute("UPDATE projects SET end_date = ? WHERE id = ?", (new_end_date, project_id))
    conn.commit()
    conn.close()

    await update.message.reply_text("Конечная дата строительства объекта обновлена.")
    return ConversationHandler.END

# Обработчик отмены
async def cancel(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    await update.message.reply_text("Редактирование проекта отменено.", reply_markup=ReplyKeyboardRemove())
    return ConversationHandler.END

# Настройка ConversationHandler для команды /update_project
update_project_handler = ConversationHandler(
    entry_points=[CommandHandler("update_project", update_project_start)],
    states={
        SELECT_PROJECT: [MessageHandler(filters.TEXT & ~filters.COMMAND, select_project)],
        UPDATE_CHOICE: [MessageHandler(filters.TEXT & ~filters.COMMAND, update_project_choice)],
        UPDATE_NAME: [MessageHandler(filters.TEXT & ~filters.COMMAND, update_project_name)],
        UPDATE_DESCRIPTION: [MessageHandler(filters.TEXT & ~filters.COMMAND, update_project_description)],
        UPDATE_LOCATION: [MessageHandler(filters.TEXT & ~filters.COMMAND, update_project_location)],
        UPDATE_START_DATE: [MessageHandler(filters.TEXT & ~filters.COMMAND, update_project_start_date)],
        UPDATE_END_DATE: [MessageHandler(filters.TEXT & ~filters.COMMAND, update_project_end_date)],
    },
    fallbacks=[CommandHandler("cancel", cancel)]
)

## Реализация команды ```/delete_project```

In [7]:
# Define dialog states
SELECT_PROJECT_TO_DELETE, CONFIRM_DELETE = range(2)

# Define the function for the /delete_project command
async def delete_project_start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    user_id = update.effective_user.id

    # Get the list of the user's projects
    conn = sqlite3.connect("projects.db")
    cursor = conn.cursor()
    cursor.execute("SELECT id, name FROM projects WHERE user_id = ?", (user_id,))
    projects = cursor.fetchall()
    conn.close()

    # Check if the user has any projects
    if not projects:
        await update.message.reply_text("У вас пока нет проектов для удаления ;(")
        return ConversationHandler.END

    # Create a keyboard with project names for selection
    project_names = [project[1] for project in projects]
    context.user_data['project_ids'] = {project[1]: project[0] for project in projects}
    keyboard = [[name] for name in project_names]

    await update.message.reply_text(
        "Выберите проект, который хотите удалить:",
        reply_markup=ReplyKeyboardMarkup(keyboard, one_time_keyboard=True)
    )
    return SELECT_PROJECT_TO_DELETE

# Handle project selection for deletion
async def select_project_to_delete(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    project_name = update.message.text
    project_id = context.user_data['project_ids'].get(project_name)

    if not project_id:
        await update.message.reply_text("Проект не найден. Пожалуйста, выберите проект из списка.")
        return SELECT_PROJECT_TO_DELETE

    context.user_data['project_id'] = project_id
    await update.message.reply_text(
        f"Вы уверены, что хотите удалить проект '{project_name}'? Введите 'yes' для подтверждения или 'no' для отмены.",
        reply_markup=ReplyKeyboardRemove()
    )
    return CONFIRM_DELETE

# Handle delete confirmation
async def confirm_delete(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    confirmation = update.message.text.lower()
    project_id = context.user_data['project_id']

    if confirmation == 'yes':
        # Delete the project from the database
        conn = sqlite3.connect("projects.db")
        cursor = conn.cursor()
        cursor.execute("DELETE FROM projects WHERE id = ?", (project_id,))
        conn.commit()
        conn.close()

        await update.message.reply_text("Проект успешно удален.")
    else:
        await update.message.reply_text("Удаление проекта отменено.")

    return ConversationHandler.END

# Обработчик отмены
async def cancel(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    await update.message.reply_text("Удаление проекта отменено.", reply_markup=ReplyKeyboardRemove())
    return ConversationHandler.END

# Создание обработчика для команды /delete_project с диалогом
delete_project_handler = ConversationHandler(
    entry_points=[CommandHandler("delete_project", delete_project_start)],
    states={
        SELECT_PROJECT_TO_DELETE: [MessageHandler(filters.TEXT & ~filters.COMMAND, select_project_to_delete)],
        CONFIRM_DELETE: [MessageHandler(filters.TEXT & ~filters.COMMAND, confirm_delete)],
    },
    fallbacks=[CommandHandler("cancel", cancel)]
)

## Реализация команды ```/show_project_details```

In [8]:
# Define dialog state
SELECT_PROJECT_TO_VIEW = range(1)

# Define the function for the /show_project_details command
async def show_project_details_start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    user_id = update.effective_user.id

    # Get the list of the user's projects
    conn = sqlite3.connect("projects.db")
    cursor = conn.cursor()
    cursor.execute("SELECT id, name FROM projects WHERE user_id = ?", (user_id,))
    projects = cursor.fetchall()
    conn.close()

    # Check if the user has any projects
    if not projects:
        await update.message.reply_text("У вас пока нет проектов для просмотра ;(")
        return ConversationHandler.END

    # Create a keyboard with project names for selection
    project_names = [project[1] for project in projects]
    context.user_data['project_ids'] = {project[1]: project[0] for project in projects}
    keyboard = [[name] for name in project_names]

    await update.message.reply_text(
        "Выберите проект, который хотите просмотреть:",
        reply_markup=ReplyKeyboardMarkup(keyboard, one_time_keyboard=True)
    )
    return SELECT_PROJECT_TO_VIEW

# Handle project selection to view details
async def select_project_to_view(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    project_name = update.message.text
    project_id = context.user_data['project_ids'].get(project_name)

    if not project_id:
        await update.message.reply_text("Проект не найден. Пожалуйста, выберите проект из списка.")
        return SELECT_PROJECT_TO_VIEW

    context.user_data['project_id'] = project_id

    # Fetch project details from the database
    conn = sqlite3.connect("projects.db")
    cursor = conn.cursor()
    cursor.execute("SELECT name, description, location, start_date, end_date FROM projects WHERE id = ?", (project_id,))
    project_details = cursor.fetchone()
    conn.close()

    if project_details:
        name, description, location, start_date, end_date = project_details
        project_info = (
            f"Название проекта: {name}\n"
            f"Описание: {description}\n"
            f"Локация: {location}\n"
            f"Дата начала: {start_date}\n"
            f"Дата окончания: {end_date}"
        )
        await update.message.reply_text(project_info)
    else:
        await update.message.reply_text("Не удалось получить информацию о проекте.")

    return ConversationHandler.END

# Обработчик отмены
async def cancel(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    await update.message.reply_text("Просмотр проекта отменен.", reply_markup=ReplyKeyboardRemove())
    return ConversationHandler.END

# Создание обработчика для команды /show_project_details с диалогом
show_project_details_handler = ConversationHandler(
    entry_points=[CommandHandler("show_project_details", show_project_details_start)],
    states={
        SELECT_PROJECT_TO_VIEW: [MessageHandler(filters.TEXT & ~filters.COMMAND, select_project_to_view)],
    },
    fallbacks=[CommandHandler("cancel", cancel)]
)

## Реализация команды ```/export_project```

In [9]:
# Define dialog state
SELECT_PROJECT_TO_EXPORT = range(1)

# Define the function for the /export_project command
async def export_project_start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    user_id = update.effective_user.id

    # Get the list of the user's projects
    conn = sqlite3.connect("projects.db")
    cursor = conn.cursor()
    cursor.execute("SELECT id, name FROM projects WHERE user_id = ?", (user_id,))
    projects = cursor.fetchall()
    conn.close()

    # Check if the user has any projects
    if not projects:
        await update.message.reply_text("У вас пока нет проектов для экспорта ;(")
        return ConversationHandler.END

    # Create a keyboard with project names for selection
    project_names = [project[1] for project in projects]
    context.user_data['project_ids'] = {project[1]: project[0] for project in projects}
    keyboard = [[name] for name in project_names]

    await update.message.reply_text(
        "Выберите проект, который хотите экспортировать:",
        reply_markup=ReplyKeyboardMarkup(keyboard, one_time_keyboard=True)
    )
    return SELECT_PROJECT_TO_EXPORT

# Handle project selection for export
async def select_project_to_export(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    project_name = update.message.text
    project_id = context.user_data['project_ids'].get(project_name)

    if not project_id:
        await update.message.reply_text("Проект не найден. Пожалуйста, выберите проект из списка.")
        return SELECT_PROJECT_TO_EXPORT

    context.user_data['project_id'] = project_id

    # Fetch full project details from the database
    conn = sqlite3.connect("projects.db")
    cursor = conn.cursor()
    cursor.execute("""
        SELECT id, name, description, location, start_date, end_date
        FROM projects WHERE id = ?
    """, (project_id,))
    project_details = cursor.fetchone()
    conn.close()

    if project_details:
        project_id, name, description, location, start_date, end_date = project_details
        
        # Create a new Excel workbook and sheet
        wb = openpyxl.Workbook()
        ws = wb.active
        ws.title = f"Информация о проекте"

        # Add headers to the Excel file
        headers = ["Поле", "Значение"]
        ws.append(headers)

        # Add full project details to the sheet
        project_info = [
            ("ID проекта", project_id),
            ("Название", name),
            ("Описание", description),
            ("Локация", location),
            ("Дата начала", start_date),
            ("Дата окончания", end_date),
        ]
        
        for info in project_info:
            ws.append(info)

        # Save the file to a BytesIO object
        excel_file = BytesIO()
        wb.save(excel_file)
        excel_file.seek(0)

        # Send the file to the user
        await update.message.reply_document(
            document=excel_file,
            filename=f"{name}_project_details.xlsx",
            caption="Вот ваш проект в формате Excel."
        )
    else:
        await update.message.reply_text("Не удалось получить информацию о проекте.")

    return ConversationHandler.END

# Обработчик отмены
async def cancel(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    await update.message.reply_text("Экспорт проекта отменен.", reply_markup=ReplyKeyboardRemove())
    return ConversationHandler.END

# Создание обработчика для команды /export_project с диалогом
export_project_handler = ConversationHandler(
    entry_points=[CommandHandler("export_project", export_project_start)],
    states={
        SELECT_PROJECT_TO_EXPORT: [MessageHandler(filters.TEXT & ~filters.COMMAND, select_project_to_export)],
    },
    fallbacks=[CommandHandler("cancel", cancel)]
)

## Настройка бота и его запуск

In [ ]:
# Creates an SQLite database if it's not created yet
def create_database():
    conn = sqlite3.connect('projects.db')
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS projects (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER,
            name TEXT NOT NULL,
            description TEXT,
            location TEXT,
            start_date TEXT,
            end_date TEXT)""")
    
    conn.commit()
    conn.close()

# Setting up our bot
TOKEN = 'YOUR-TOKEN-API'

application = Application.builder().token(TOKEN).build()

# Command handlers
application.add_handler(CommandHandler("start", start)) # /start
application.add_handler(conversation_handler) # /add_project
application.add_handler(CommandHandler("list_projects", list_projects)) # /list_projects
application.add_handler(update_project_handler) # /update_project
application.add_handler(delete_project_handler) # /delete_project
application.add_handler(show_project_details_handler) # /show_project_details
application.add_handler(export_project_handler) # /export_project

# Create a database if it's not created yet
create_database()

# Run the bot
application.run_polling()